In [3]:
# runs 2nd level analyses for PPI and FC
import sys
sys.path.insert(0, '/user_data/csimmon2/git_repos/ptoc')
import glob
import pandas as pd
import gc
from nilearn import image, input_data
import numpy as np
import nibabel as nib
import os
from nilearn.glm.first_level import compute_regressor
import warnings
import ptoc_params as params
import argparse

raw_dir = params.raw_dir
results_dir = params.results_dir
warnings.filterwarnings('ignore')

# Set up run combinations like obj script
run_num = 2  # Adjust based on your data
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) 
              for rn2 in range(rn1 + 1, run_num + 1)]
tr = 1
vols = 341

def extract_roi_timeseries(img, roi_mask, hemisphere='left'):
    """Extract ROI timeseries using parcels and hemisphere masking."""
    roi_data = roi_mask.get_fdata()
    mid_x = roi_data.shape[0] // 2
    
    hemi_mask = np.zeros_like(roi_data)
    if hemisphere == 'left':
        hemi_mask[:mid_x, :, :] = 1
    else:
        hemi_mask[mid_x:, :, :] = 1
    
    combined_mask = roi_data * hemi_mask
    mask_img = nib.Nifti1Image(combined_mask, roi_mask.affine)
    
    roi_masker = input_data.NiftiMasker(mask_img, standardize=True)
    seed_time_series = roi_masker.fit_transform(img)
    return np.mean(seed_time_series, axis=1).reshape(-1, 1)

def make_psy_cov(runs, ss, temp_dir):
    """Create psychological covariates for specified runs."""
    cov_dir = f'{temp_dir}/covs'
    times = np.arange(0, vols * len(runs), tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for run in runs:
        ss_num = ss.split('-')[1].replace('spaceloc', '')
        tool_cov_file = f'{cov_dir}/ToolLoc_spaceloc{ss_num}_run{run}_tool.txt'
        nontool_cov_file = f'{cov_dir}/ToolLoc_spaceloc{ss_num}_run{run}_non_tool.txt'

        if os.path.exists(tool_cov_file) and os.path.exists(nontool_cov_file):
            tool_cov = pd.read_csv(tool_cov_file, sep='\t', header=None, 
                                 names=['onset', 'duration', 'value'])
            nontool_cov = pd.read_csv(nontool_cov_file, sep='\t', header=None, 
                                    names=['onset', 'duration', 'value'])
            nontool_cov['value'] *= -1

            # Adjust onsets for concatenated runs
            run_offset = vols * runs.index(run)
            tool_cov['onset'] += run_offset
            nontool_cov['onset'] += run_offset
            
            full_cov = pd.concat([full_cov, tool_cov, nontool_cov])

    full_cov = full_cov.sort_values(by=['onset'])
    cov = full_cov.to_numpy()
    psy, _ = compute_regressor(cov.T, 'spm', times)
    return psy

def conduct_analyses(sub, rois=['LO', 'pIPS']):
    import glob  # Add at top
    
    temp_dir = f'{raw_dir}/{sub}/ses-01/derivatives/fsl/toolloc'
    roi_dir = f'{raw_dir}/{sub}/ses-01/derivatives/rois'
    out_dir = f'/user_data/csimmon2/temp_derivatives/{sub}/ses-01/derivatives'
    os.makedirs(f'{out_dir}/fc', exist_ok=True)
    
    mask_path = f'{raw_dir}/{sub}/ses-01/anat/{sub}_ses-01_T1w_brain_mask.nii.gz'
    whole_brain_mask = nib.load(mask_path)
    brain_masker = input_data.NiftiMasker(whole_brain_mask, standardize=True)

    run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) 
                  for rn2 in range(rn1 + 1, run_num + 1)]

    for rr in rois:
        roi_path = f'{roi_dir}/parcels/{rr}.nii.gz'
        roi_img = nib.load(roi_path)
        
        for hemi in ['left', 'right']:
            print(f"Processing ROI: {rr}, Hemisphere: {hemi}")
            
            fc_file = f'{out_dir}/fc/{sub}_{rr}_{hemi}_toolloc_fc_native.nii.gz'
            ppi_file = f'{out_dir}/fc/{sub}_{rr}_{hemi}_toolloc_ppi_native.nii.gz'
            
            do_fc = not os.path.exists(fc_file)
            do_ppi = not os.path.exists(ppi_file)
            
            if not do_fc and not do_ppi:
                print(f'Both FC and PPI files exist for {rr} {hemi}. Skipping...')
                continue
            
            for rc in run_combos:
                print(f"Processing run combination: {rc}")
                
                filtered_list = []
                for run in rc:
                    curr_run = image.load_img(
                        f'{temp_dir}/run-0{run}/1stLevel.feat/filtered_func_data_reg.nii.gz')
                    curr_run = image.clean_img(curr_run, standardize=True)
                    filtered_list.append(curr_run)

                img4d = image.concat_imgs(filtered_list)
                phys = extract_roi_timeseries(img4d, roi_img, hemisphere=hemi)
                brain_time_series = brain_masker.fit_transform(img4d)
                
                if do_fc:
                    correlations = np.dot(brain_time_series.T, phys) / phys.shape[0]
                    correlations = np.arctanh(correlations)
                    correlations = correlations.reshape(1, -1)
                    correlation_img = brain_masker.inverse_transform(correlations)
                    temp_fc_file = f'{out_dir}/fc/{sub}_{rr}_{hemi}_run{rc[0]}{rc[1]}_fc_temp.nii.gz'
                    nib.save(correlation_img, temp_fc_file)
                
                if do_ppi:
                    psy = make_psy_cov(rc, sub, f'{raw_dir}/{sub}/ses-01')
                    
                    min_length = min(psy.shape[0], phys.shape[0])
                    psy = psy[:min_length]
                    phys = phys[:min_length]
                    brain_time_series = brain_time_series[:min_length]
                    
                    ppi = psy * phys
                    correlations = np.dot(brain_time_series.T, ppi) / ppi.shape[0]
                    correlations = np.arctanh(correlations)
                    correlations = correlations.reshape(1, -1)
                    correlation_img = brain_masker.inverse_transform(correlations)
                    temp_ppi_file = f'{out_dir}/fc/{sub}_{rr}_{hemi}_run{rc[0]}{rc[1]}_ppi_temp.nii.gz'
                    nib.save(correlation_img, temp_ppi_file)
                
                del img4d, phys, brain_time_series, correlations, correlation_img
                if 'ppi' in locals(): del ppi
                gc.collect()
            
            # After processing all run combinations, average temp files
            if do_fc:
                temp_fc_files = sorted(glob.glob(f'{out_dir}/fc/{sub}_{rr}_{hemi}_run*_fc_temp.nii.gz'))
                mean_fc = image.mean_img(temp_fc_files)
                nib.save(mean_fc, fc_file)
                for f in temp_fc_files: os.remove(f)
                print(f'Saved FC result for {rr} {hemi}')
            
            if do_ppi:
                temp_ppi_files = sorted(glob.glob(f'{out_dir}/fc/{sub}_{rr}_{hemi}_run*_ppi_temp.nii.gz'))
                mean_ppi = image.mean_img(temp_ppi_files)
                nib.save(mean_ppi, ppi_file)
                for f in temp_ppi_files: os.remove(f)
                print(f'Saved PPI result for {rr} {hemi}')

if __name__ == "__main__":
    rois = ['LO', 'pIPS']
    sub = 'sub-spaceloc1007'
    conduct_analyses(sub, rois)

Processing ROI: LO, Hemisphere: left
Both FC and PPI files exist for LO left. Skipping...
Processing ROI: LO, Hemisphere: right
Both FC and PPI files exist for LO right. Skipping...
Processing ROI: pIPS, Hemisphere: left
Processing run combination: [1, 2]
Saved FC result for pIPS left
Saved PPI result for pIPS left
Processing ROI: pIPS, Hemisphere: right
Processing run combination: [1, 2]


MemoryError: Unable to allocate 16.2 GiB for an array with shape (341, 12779520) and data type float32

In [1]:
# clear memory: although you usually have to restart the kernel to really clear the memory
import gc
gc.collect()  # Force garbage collection

5

In [2]:
# check memory usage
import psutil
process = psutil.Process()
print(f"Memory used by Python: {process.memory_info().rss / 1024**3:.2f} GB")
print(f"Total system memory used: {psutil.virtual_memory().percent}%")

Memory used by Python: 0.05 GB
Total system memory used: 3.2%
